In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import re
from tqdm import tqdm
from hanspell import spell_checker

# 네이버 지식인 데이터 전처리

### 1. 강아지 우울증 내용 삭제(수작업)

In [ ]:
kin[kin['title'].str.contains('강아지')]
kin = kin.drop([35,36,37,38,39])

### 2. 의사 답변만 남기기

In [ ]:
keyword = ['상담','임상','의사']
filtered_kin = kin[kin['answer'].str.contains('|'.join(keyword))]
filtered_kin.reset_index(drop=True, inplace=True)
filtered_kin

In [ ]:
filtered_kin.to_csv('naver_kin_filtered.csv',index=False)
filtered_kin.to_excel('naver_kin_filtered.xlsx',index=False)

### 3. openai api이용해서 answer에서 필요한 답변만 추출

In [ ]:
# !pip install openai

In [ ]:
filtered_kin = pd.read_csv('naver_kin_filtered.csv')

In [ ]:
import openai

# OpenAI API 키 설정 (자신의 API 키로 대체해야 함)
api_key = ''
openai.api_key = api_key

# GPT-3 API 호출 함수
def generate_text(text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [
            {"role": "system", "content": "입력받은 문장은, 전문의들이 우울증이 있는 사람들에게 답변한 문장들이다."},
            {"role": "system", "content": "우울증에 대한 해결책이 포함된 부분만 요약해서 문장으로 제시해라"},
            {"role": "system", "content": "한국어로 답변해라"},
            {"role": "system", "content": "입력받은 문장의 문체를 최대한 유지한 채로 문장을 제시해라."},
            {"role": "system", "content": "단순히 해결책에 대해 나열하지 말고, 연속된 문장들로 답변해라."},
            {"role": "user", "content" : text}
        ],
        temperature=0.5
    )
    return response.choices[0].message.content

In [ ]:
### 한번에 많은 요청을 보내면 오류남
#api 토큰 제한이 4000자라서 4000자로 제한
max_length = 3000
filtered_kin['answer_cut'] = filtered_kin['answer'].apply(lambda x: x[:max_length])
filtered_kin['processed_answer'] = filtered_kin['answer_cut'].apply(generate_text)
filtered_kin.to_csv('naver_kin_filtered_processed.csv', index=False)
filtered_kin

In [ ]:
### 위 코드 에러 발생 시 실행
# 데이터프레임을 100개씩 묶어서 처리
max_length = 3000
batch_size = 300
num_batches = len(filtered_kin) // batch_size + 1

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = filtered_kin.iloc[start_idx:end_idx]

    # 최대 길이로 자르고 텍스트 처리
    batch_data['answer_cut'] = batch_data['answer'].apply(lambda x: x[:max_length])
    batch_data['processed_answer'] = batch_data['answer_cut'].apply(generate_text)

    # 중간 결과 출력
    print(f"Batch {batch_num + 1}/{num_batches} processed.")

    # 중간 결과 저장
    intermediate_output_path = f'intermediate_output_batch_{batch_num + 1}.csv'
    batch_data.to_csv(intermediate_output_path, index=False)

# 최종 결과 출력
print("Processing completed.")

batch1 = pd.read_csv('intermediate_output_batch_1.csv')
batch2 = pd.read_csv('intermediate_output_batch_2.csv')
batch3 = pd.read_csv('intermediate_output_batch_3.csv')
batch4 = pd.read_csv('intermediate_output_batch_4.csv')

# 데이터프레임 합치기
combined_df = pd.concat([batch1, batch2, batch3, batch4], ignore_index=True)
combined_df.to_csv('naver_kin_filtered_processed.csv', index=False)
combined_df.to_excel('naver_kin_filtered_processed.xlsx', index=False)

### 4. 질문, 답변에 대한 전처리
- 데이터 증강전

In [ ]:
df = pd.read_csv('naver_kin_filtered_processed.csv')
#df = pd.read_excel('total.xlsx', index_col = 0)
df

,title,question,answer,url,answer_cut,processed_answer
0,우울증 테스트,안녕하세요. 저는 예비 중1입니다. 일단 우울증인 것 같기는 한데 아직 잘 모르겠어...,안녕하세요. 대한한의사협회-네이버 지식iN 상담한의사 전창환 입니다.​간이로 하는...,https://kin.naver.com/qna/detail.naver?d1id=7&...,안녕하세요. 대한한의사협회-네이버 지식iN 상담한의사 전창환 입니다.​간이로 하는...,우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....
1,우울증 극복 (청주 우울증),청주 30대 후반/여 우울증우울증 극복 어떻하죠 남들하고 비교하다보니 자존감이 너무...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 변형남입니다.말씀주신 여러가지 불편한 ...",https://kin.naver.com/qna/detail.naver?d1id=7&...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 변형남입니다.말씀주신 여러가지 불편한 ...","우울증 극복 방법은 다양하게 있지만, 우선적으로 우울에 대한 감정과 우울증에 대한 ..."
2,아이가 우울증 같아요. (잠실 우울증),잠실 10대 초반/여 우울증아이가 10살인데 우울증 증상을 보입니다.처음에는 의심되...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 이유진입니다.자녀분의 증상으로 걱정이 ...",https://kin.naver.com/qna/detail.naver?d1id=7&...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 이유진입니다.자녀분의 증상으로 걱정이 ...",우울증을 겪는 아이들을 치료하기 위해서는 안전하고 효과적인 방법을 사용해야 합니다....
3,청소년 우울증 상담 받을까요?(),40대 중반/여청소년 우울증 상담 질문드리고 싶어요... 아이가 지금 청소년 우울증...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 김현우입니다.청소년 우울증 상담과 사프...",https://kin.naver.com/qna/detail.naver?d1id=7&...,"안녕하세요, 닥톡-네이버 지식iN 상담한의사 김현우입니다.청소년 우울증 상담과 사프...","청소년 우울증에 대한 해결책으로는 다음과 같은 접근 방법을 권장드립니다. 첫째로, ..."
4,초6 이거 우울증인가요..,초딩 졸업을 앞두고 있는 학생인데요가정사도 안 좋아지고 정말 사랑하는 남친이랑 싸우...,안녕하세요.언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스트레스 ...,https://kin.naver.com/qna/detail.naver?d1id=7&...,안녕하세요.언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스트레스 ...,"안녕하세요. 우선, 언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스..."
...,...,...,...,...,...,...
900,우울증이려나요?,현재 중3 여학생입니다 요즘 울음을 많이 터트리는데 어제도 자신있던 과목에서 실수하...,​질문내공50우울증이려나요?현재 중3 여학생입니다 요즘 울음을 많이 터트리는데 어...,https://kin.naver.com/qna/detail.naver?d1id=7&...,​질문내공50우울증이려나요?현재 중3 여학생입니다 요즘 울음을 많이 터트리는데 어...,우울증은 감정이 격해지는 것 외에도 다른 부분들은 평소와 같이 행동한다는 점을 강조...
901,우울증 예방방안,우울증 예방법 좀 알려주세요..그리고 그 예방법이 왜 예방법인지도 …,우울증의 원인을 알고 그 원인 요소가 생활에 발생되지 않게 하면 우울증은 생길 수...,https://kin.naver.com/qna/detail.naver?d1id=7&...,우울증의 원인을 알고 그 원인 요소가 생활에 발생되지 않게 하면 우울증은 생길 수...,우울증의 원인은 생활에서 발생하는 요소들입니다. 이러한 요소들을 제거하면 우울증을 ...
902,우울증질문,나 자신이 우울증인 것 같다는 생각이 들면우울증인걸까요..?평소에 조울증처럼 기분도...,나 자신이 우울증인 것 같다 라는 생각만으로 우울증이라고 판단을 내릴 순 없어요실...,https://kin.naver.com/qna/detail.naver?d1id=7&...,나 자신이 우울증인 것 같다 라는 생각만으로 우울증이라고 판단을 내릴 순 없어요실...,"우선, 자기 자신이 우울증인지 아닌지 판단하는 것은 쉽지 않습니다. 그러므로, 현재..."
903,이거 우울증인가요? ( 내공+30 ),어두워 지고 방에 혼자 있으면자꾸 우울해지곤 합니다 요새는 불안하고 심장이 잘못한 ...,안녕하세요? 정신건강임상심리사1급/임상심리전문가 해말근이쓴입니다.​질문자님 마음의...,https://kin.naver.com/qna/detail.naver?d1id=7&...,안녕하세요? 정신건강임상심리사1급/임상심리전문가 해말근이쓴입니다.​질문자님 마음의...,"우선, 안녕하세요. 저는 정신건강임상심리사1급/임상심리전문가 해말근이라고 합니다. ..."


- 데이터 증강 후

In [ ]:
#df = pd.read_csv('naver_kin_filtered_processed.csv')
df = pd.read_excel('total.xlsx', index_col = 0)
df

,Q,A,label
0,저는 예비 중1입니다. 일단 우울증인 것 같기는 한데 아직 잘 모르겠어서요. 우울증...,우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....,original
1,청주 30대 후반/여 우울증우울증 극복 어떻하죠 남들하고 비교하다보니 자존감이 너무...,"우울증 극복 방법은 다양하게 있지만, 우선적으로 우울에 대한 감정과 우울증에 대한 ...",original
2,잠실 10대 초반/여 우울증아이가 10살인데 우울증 증상을 보입니다.처음에는 의심되...,우울증을 겪는 아이들을 치료하기 위해서는 안전하고 효과적인 방법을 사용해야 합니다....,original
3,40대 중반/여청소년 우울증 상담 질문드리고 싶어요... 아이가 지금 청소년 우울증...,"청소년 우울증에 대한 해결책으로는 다음과 같은 접근 방법을 권장드립니다. 첫째로, ...",original
4,초딩 졸업을 앞두고 있는 학생인데요가정사도 안 좋아지고 정말 사랑하는 남친이랑 싸우...,"안녕하세요. 우선, 언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스...",original
...,...,...,...
2693,현재 중3 여학생입니다 요즘 울음을 많이 터트리는데 어제도 자신있던 과목에서 실수하...,우울증은 감정이 격해지는 것과 다른 부분들은 다 같이 행동한다는 점을 강조하셨습니다...,insert
2694,우울증 예방법 좀 알려주세요..그리고 그 예방법이 왜 예방법인지도 …,"우울증의 원인은 생활에서 해로운 요소들입니다. 이러한 요소들을 제거하면서 줄일 수,...",insert
2695,나 자신이 우울증인 것 같다는 생각이 들면우울증인걸까요..?평소에 조울증처럼 기분도...,"우선, 자기 자신이 스스로 판단하는 것은 하지만 그러므로, 현재 알고 있는 감정이 ...",insert
2696,어두워 지고 방에 혼자 있으면자꾸 우울해지곤 합니다 요새는 불안하고 심장이 잘못한 ...,"우선, 안녕하세요. 저는 정신과 / 임상심리전문가 해말근이라고 알려진 질문자님의 마...",insert


In [ ]:
# 데이터 전처리 함수
def update_question(row):
    if row['question'] == '제목과 내용 동일':
        return row['title']
    else:
        return row['question']

def replace_text(text):
    # 초성 줄임말을 단어로 반환
    text = text.replace('ㅈㅅ', '자살')
    text = text.replace('ㅈㅎ', '자해')
    return text

def remove_words_hello(text):
    # 문장을 공백을 기준으로 단어로 분할
    words = text.split()

    # '안녕'을 포함하는 단어를 제거
    filtered_words = [word for word in words if '안녕' not in word]

    # 제거된 단어들을 다시 문자열로 결합
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

def text_normalization(text):
    # 정규표현식을 사용하여 괄호와 괄호 내의 내용을 삭제
    cleaned_text = re.sub(r'\([^)]*\)', '', text)
    # 자음, 모음으로만 이뤄진 글자 삭제
    cleaned_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', cleaned_text)
    return cleaned_text

def ans_normalization(text):
    # '\n\n'을 빈 문자열로 대체
    return text.replace('\n\n', '').strip()

# apply 함수를 사용하여 update_question 함수를 적용
tqdm.pandas()

# total_xlsx 사용할 때는 생략 가능
# df.rename(columns={'question' : 'Q','processed_answer' : 'A'}, inplace=True)
# df = df[['Q','A']]

df['Q'] = df['Q'].progress_apply(replace_text)
df['Q'] = df['Q'].progress_apply(remove_words_hello)
df['Q'] = df['Q'].progress_apply(text_normalization)
df['A'] = df['A'].progress_apply(ans_normalization)
df['A'] = df['A'].progress_apply(text_normalization)
df['A'] = df['A'].progress_apply(remove_words_hello)

df = df[~df['A'].str.contains('채택')]
df = df[~df['A'].str.contains('http')]
df = df[~df['A'].str.contains('cyber1388')]
df = df[~df['A'].str.contains('마음애')]
df = df[~df['A'].str.contains('광주심리상담')]
df.reset_index(drop=True, inplace=True)
df

100%|███████████████████████████████████████████████████████████████████████████| 2698/2698 [00:00<00:00, 25843.58it/s]


,Q,A,label
0,저는 예비 중1입니다. 일단 우울증인 것 같기는 한데 아직 잘 모르겠어서요. 우울증...,우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....,original
1,청주 30대 후반/여 우울증우울증 극복 어떻하죠 남들하고 비교하다보니 자존감이 너무...,"우울증 극복 방법은 다양하게 있지만, 우선적으로 우울에 대한 감정과 우울증에 대한 ...",original
2,잠실 10대 초반/여 우울증아이가 10살인데 우울증 증상을 보입니다.처음에는 의심되...,우울증을 겪는 아이들을 치료하기 위해서는 안전하고 효과적인 방법을 사용해야 합니다....,original
3,40대 중반/여청소년 우울증 상담 질문드리고 싶어요... 아이가 지금 청소년 우울증...,"청소년 우울증에 대한 해결책으로는 다음과 같은 접근 방법을 권장드립니다. 첫째로, ...",original
4,초딩 졸업을 앞두고 있는 학생인데요가정사도 안 좋아지고 정말 사랑하는 남친이랑 싸우...,"우선, 언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스트레스 상황으...",original
...,...,...,...
2445,현재 중3 여학생입니다 요즘 울음을 많이 터트리는데 어제도 자신있던 과목에서 실수하...,우울증은 감정이 격해지는 것과 다른 부분들은 다 같이 행동한다는 점을 강조하셨습니다...,insert
2446,우울증 예방법 좀 알려주세요..그리고 그 예방법이 왜 예방법인지도 …,"우울증의 원인은 생활에서 해로운 요소들입니다. 이러한 요소들을 제거하면서 줄일 수,...",insert
2447,나 자신이 우울증인 것 같다는 생각이 들면우울증인걸까요..?평소에 조울증처럼 기분도...,"우선, 자기 자신이 스스로 판단하는 것은 하지만 그러므로, 현재 알고 있는 감정이 ...",insert
2448,어두워 지고 방에 혼자 있으면자꾸 우울해지곤 합니다 요새는 불안하고 심장이 잘못한 ...,"우선, 저는 정신과 / 임상심리전문가 해말근이라고 알려진 질문자님의 마음의 감기 다...",insert


In [ ]:
df.to_csv('final_df_naver_kin.csv', index=False)

In [ ]:
df.to_csv('naver_kin_filtered_processed_v2.csv', index=False)

## 데이터셋 답변에 대한 추가 전처리 진행
- 채택해주세요 등의 문구가 포함된 광고성 답변 삭제
- \n\n등의 개행문구 삭제
- https://~ 등의 링크 제공하는 답변도 삭제
- 1388 권유 삭제


### final_df_piece.csv : 5170개 랜덤하게 추출하여 사용
### final_df_total.csv : 29667 전체 사용

In [ ]:
pc = pd.read_csv('final_df_piece.csv')
tt = pd.read_csv('final_df_total.csv')

In [ ]:
tt['A']

0        우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....
1        우울증 극복 방법은 다양하게 있지만, 우선적으로 우울에 대한 감정과 우울증에 대한 ...
2        우울증을 겪는 아이들을 치료하기 위해서는 안전하고 효과적인 방법을 사용해야 합니다....
3        청소년 우울증에 대한 해결책으로는 다음과 같은 접근 방법을 권장드립니다. 첫째로, ...
4        우선, 언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스트레스 상황으...
                               ...                        
29662                                건강에 대한 걱정 때문에 속상하시군요.
29663                   경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.
29664                건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.
29665                        노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.
29666                                 가족과의 문제 때문에 속상하시겠어요.
Name: A, Length: 29667, dtype: object

In [ ]:
tt[tt['A'].str.contains('[a-zA-Z]')].iloc[102,1]

'안녕하세요! 식욕이 떨어지고 우울한 기분 줄이기 위해, 좋은 음식을 알고 계시다니 힘들어 보이시네요. 하지만 걱정하지 마세요, 우울증 완화에 도움을 줄 수, 몇 가지 음식을 소개해 드릴게요. 먼저 오메가 - 3 지방산 함유가 좋아요. 오메가 - 3는 뇌 기능을 촉진시켜 우울증 증상 완화에 도움 줄 수, 연어, 참치와 같은 물고기와 호두, 아몬드 같은 견과류를 섭취하시면 좋아요. 두번째로, 엽산이 들어간 음식 섭취해보세요. 엽산은 세로토닌이라는 신경 전달물질의 수치를 안정시켜 우울감을 줄일 수 있어요. 그리고 아스파라거스, 겉귀리와 같은 음식들이 엽산이 풍부한 음식이에요. 그리고 비타민 B6와 B12 함유 식품도 좋아요. 비타민 B6와 B12는 우울기분 개선에 도움 주는 중요한 영양소입니다. 가금류, 그리고 우유 및 유제품, 그리고 콩류와 같은 음식에 비타민 B6와 함께 많이 함유되어 있어요. 또한, 영양제를 고려하신다면 유어닥터 엘나스 닥터베스트와 같은 제품들은 전문가의 의견을 권유하고, 자세히 살펴볼 것이 좋아요. 음식이나, 개인의 알레르기 반응에 따라 다를 수 있으니 전문가의 조언을 받는 것이 가장 좋아요. 음식을 섭취할 때는 영양가를 높일 제한 사항이 있는지 알아보는 것도 중요해요. 또한 정기적인 운동, 정신과 이성 간의 대화 등 우울감을 완화시킬 수 있는 다양한 방법들도 함께 고려해보세요. 물론 전부일 수는 없지만 참고해주시길 바라며, 좋은 결과 얻을 수 있기를 바랍니다. 건강하게 지내세요!'

In [ ]:
tt[tt['A'].str.contains('님')]

,Q,A
0,저는 예비 중1입니다. 일단 우울증인 것 같기는 한데 아직 잘 모르겠어서요. 우울증...,우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....
5,요즘 힘든일이 많아서 떠돌아 다니는 우울증 테스트를 해봤는데 결과가 심각하더라고요 ...,우울증 자가테스트를 진단해보셨군요. 증상이 심각하다면 꼭 주변 사람들에게 알리고 도...
9,기존에 우울증약을 밤에 먹고 adhd약을 아침에 먹고 있는 중인데 우울증약을 밤에 ...,안녕하세요. 정신건강의학과 전문의 이준협입니다. 우울증 약과 ADHD 약을 복용 중...
11,요즘 제가 좀 많이 무기력해지고 잠을 잘 못자고 밥을 잘 안 먹고 주변에서 평소와 ...,"비공개님, 우울감과 우울증은 다른 것을 알려드립니다. 주신 증상들은 우울감일 가능성..."
20,제가 고2이고 현재 우울증이 있는 상태입니다 그냥 우울하고 학교가면 아무 재미가 없...,"우선, 우울증 진단을 받았다면 진단서를 제출하는 것이 좋습니다. 진단서는 의사의 의..."
...,...,...
29532,친구랑 놀다 들어왔는데 엄마가 지나치게 간섭해서 마치 내가 뭘 잘못한 기분이 들어.,부모님의 지나친 간섭으로 자책감이 드시는군요.
29554,내가 게임 중독이라 학업을 소홀히 한다고 생각하는 부모님 때문에 실패한 느낌이 들어.,부모님 때문에 실패한 느낌이 드는군요.
29595,부모님이 같이 살자고 하신 걸 거절했는데 죄책감이 들어.,부모님께서 하신 말씀을 거절하신 것 때문에 죄책감이 드시나 봐요.
29608,부모님이 공부할 시간에 연애한다고 혼을 내셔서 우울해. 내가 그렇게 큰 잘못을 한 걸까?,이성 친구 문제로 부모님과 갈등을 빚어서 속상하셨겠어요.


In [ ]:
tt

,Q,A
0,저는 예비 중1입니다. 일단 우울증인 것 같기는 한데 아직 잘 모르겠어서요. 우울증...,우울증 테스트는 신뢰할만하지 않습니다. 중요한 것은 본인의 현재 우울한 상태입니다....
1,청주 30대 후반/여 우울증우울증 극복 어떻하죠 남들하고 비교하다보니 자존감이 너무...,"우울증 극복 방법은 다양하게 있지만, 우선적으로 우울에 대한 감정과 우울증에 대한 ..."
2,잠실 10대 초반/여 우울증아이가 10살인데 우울증 증상을 보입니다.처음에는 의심되...,우울증을 겪는 아이들을 치료하기 위해서는 안전하고 효과적인 방법을 사용해야 합니다....
3,40대 중반/여청소년 우울증 상담 질문드리고 싶어요... 아이가 지금 청소년 우울증...,"청소년 우울증에 대한 해결책으로는 다음과 같은 접근 방법을 권장드립니다. 첫째로, ..."
4,초딩 졸업을 앞두고 있는 학생인데요가정사도 안 좋아지고 정말 사랑하는 남친이랑 싸우...,"안녕하세요. 우선, 언급한 내용으로 보아 우울증에 의한 것일 수도 있고 급작스런 스..."
...,...,...
27190,이제 몸이 점점 약해진다는 게 느껴져. 아내에게 미안하고 속상한 마음이 들어.,건강에 대한 걱정 때문에 속상하시군요.
27191,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.
27192,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.
27193,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.


In [ ]:
def remove_words_hello(text):
    # 문장을 공백을 기준으로 단어로 분할
    words = text.split()

    # '안녕'을 포함하는 단어를 제거
    filtered_words = [word for word in words if '안녕' not in word]

    # 제거된 단어들을 다시 문자열로 결합
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

def ans_normalization(text):
    # '\n\n'을 빈 문자열로 대체
    return text.replace('\n\n', '').strip()

tt['A'] = tt['A'].apply(ans_normalization)
tt['A'] = tt['A'].apply(remove_words_hello)
tt = tt[~tt['A'].str.contains('채택')]
tt = tt[~tt['A'].str.contains('http')]
tt = tt[~tt['A'].str.contains('cyber1388')]
tt.reset_index(drop=True, inplace=True)

pc['A'] = pc['A'].apply(ans_normalization)
pc['A'] = pc['A'].apply(remove_words_hello)
pc = pc[~pc['A'].str.contains('채택')]
pc = pc[~pc['A'].str.contains('http')]
pc = pc[~pc['A'].str.contains('cyber1388')]
pc.reset_index(drop=True, inplace=True)

In [ ]:
pc.to_csv('final_df_piece.csv', index = False)
tt.to_csv('final_df_total.csv', index = False)